In [188]:
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

In [61]:
d = 16

In [3]:
metadata = pd.read_csv("metadata.csv")
metadata.head()

,path,patient_id,series_id,frame_id,liver,spleen,right_kidney,left_kidney,bowel,aortic_hu,...,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,1523_62556_0.png,1523,62556,0,0,0,0,0,1,254,...,0,1,0,1,0,0,0,0,1,1
1,1523_62556_1.png,1523,62556,1,0,0,0,0,1,254,...,0,1,0,1,0,0,0,0,1,1
2,1523_62556_2.png,1523,62556,2,0,0,0,0,1,254,...,0,1,0,1,0,0,0,0,1,1
3,1523_62556_3.png,1523,62556,3,0,0,0,0,1,254,...,0,1,0,1,0,0,0,0,1,1
4,1523_62556_4.png,1523,62556,4,0,0,0,0,1,254,...,0,1,0,1,0,0,0,0,1,1


In [254]:
import torch
from torch.utils.data import IterableDataset, DataLoader
import torchvision.transforms as T

series = metadata['series_id'].unique()

for s in series:
    ndseries = metadata[metadata['series_id'] == s].to_numpy()
    labels, padded_ndseries = torch.from_numpy(ndseries[0, 3:].astype(np.float32)), pad_and_chunk_series(ndseries, d)
    shape = np.array(Image.open(f'images/{padded_ndseries[0][0][0]}')).shape
    print(len(padded_ndseries))
    for i, batch in enumerate(padded_ndseries):
        paths = batch[:, 0]
        im = np.array([np.array(Image.open(f'images/{p}')) if p != -1 else np.zeros(shape) for p in paths])
        im = np.expand_dims(im, 1).astype(np.float32)
        im = torch.from_numpy(im)
        print(im.shape)
        break
    break

9
torch.Size([16, 1, 512, 512])


In [255]:
class VideoSegmentationDataset(IterableDataset):
    def __init__(self, d=16, train=True, transform=None, images = 'images/', masks='masks/', metadata='metadata.csv'):
        self.d = d
        self.train = train
        self.transform = (
            transform if transform else T.Resize((512, 512), antialias=True)
        )
        self.metadata = pd.read_csv(metadata)
        self.images, self.masks = images, masks
    
    def pad_and_chunk_series(self, series, d):
        l = len(series)
        max = l+d-l%d
        chunks = [series[i:i+d] for i in range(0,max,d)]
        last = len(chunks[-1])
        if last != d:
            ls = chunks[0][0].tolist()
            c = [-1] + ls[1:3] + [-1] + [0] * 5 + ls[9:]
            chunks[-1] = np.append(chunks[-1], [c] * (d - last), axis=0)
        chunks = np.array(chunks)
        return chunks

    def generator(self):
        metadata, d = self.metadata, self.d
        series = metadata['series_id'].unique()

        for s in series:
            ndseries = metadata[metadata['series_id'] == s].to_numpy()
            labels, padded_ndseries = torch.from_numpy(ndseries[0, 3:].astype(np.float32)), self.pad_and_chunk_series(ndseries, d)
            shape = np.array(Image.open(f'{self.images}/{padded_ndseries[0][0][0]}')).shape
            for i, batch in enumerate(padded_ndseries):
                if not self.train and i != len(padded_ndseries) - 1:
                    continue
                if self.train and i == len(padded_ndseries) - 1:
                    continue
                paths = batch[:, 0]
                im = np.array([np.array(Image.open(f'{self.images}/{p}')) if p != -1 else np.zeros(shape) for p in paths])
                im = np.expand_dims(im, 1).astype(np.float32)
                im = torch.from_numpy(im)
                im = self.transform(im) / 255
                
                masks = np.array([np.array(Image.open(f'{self.masks}/{p}')) if p != -1 else np.zeros(shape) for p in paths])
                masks = np.expand_dims(masks, 1).astype(np.float32)
                masks = torch.from_numpy(masks)
                masks = self.transform(im) / 5

                yield (im, masks), labels
    
    def __iter__(self):
        return self.generator()

In [260]:
dataset = VideoSegmentationDataset(train=True)

In [261]:
for i, b in enumerate(dataset):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 